---
# 0. 라이브러리 
---

In [2]:
# 문장을 토큰화하는 모듈 설치
!pip3 install torchtext==0.10.0
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
^C
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
2023-02-08 15:33:49.663463: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2023-02-08 15:34:01.245205: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to fl

In [3]:
import os 
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
import spacy
import numpy as np

import random
import math
from tqdm.notebook import tqdm

from torchsummary import summary as summary_
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

---
# 1. Data
----

In [5]:
# 문장을 토큰화하는 모델 로드
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [6]:
# tokenizer function 생성
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

#### Field 정의

- sequential : 시퀀스 데이터 여부. (True가 기본값)
- use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
- tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
- lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
- batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
- is_target : 레이블 데이터 여부. (False가 기본값)
- fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.

In [7]:
# torchtext의 Field는 데이터를 어떻게 처리할지 조절합니다.
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

---
### 1-1. Data Load
----

In [8]:
# 30,000개의 영어, 독일, 프랑스어 문장을 포함합니다.
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC,TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 799kB/s] 


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 230kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 214kB/s]


In [9]:
print('Train 개수:', len(train_data))
print('Test 개수:', len(test_data))
print('Valid 개수:', len(valid_data))

Train 개수: 29000
Test 개수: 1000
Valid 개수: 1014


In [10]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


---
### 1-2. Vocab 생성
----

- min_freq = 해당 인수를 사용하여 n회 이상 나타내는 토큰만 어휘에 표시   
   n-1회 나타날 경우 <UNK\>으로 표시

In [11]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [12]:
print('Train Vocab의 수:', len(SRC.vocab))
print('Test Vocab의 수:', len(TRG.vocab))

Train Vocab의 수: 7853
Test Vocab의 수: 5893


In [13]:
batch_size = 128

# Iterator를 통해 원본 문장과 배치 문장과 동일하게 패딩 처리
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size=batch_size, device=device)

---
# 2. Model
----

<img src = 'https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq1.png'>


---
### 2-1. Encoder
----

<img src='https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq2.png'>

- Input_dim = 단일 벡터(입력 소스)의 크기
- emb_dim = 임베딩 레이어의 차원   
  원-핫 벡터를 emb_dimention 차원의 dense 벡터로 변환

- hid_dim = LSTM의 hidden state와 cell state의 수

- n_layer = LSTM 레이어의 수 

1. 임베딩 레이어를 사용해 dense 벡터로 변환된 소스 문장을 전달
2. Dropout에 적용
3. LSTM에 전달 

In [14]:
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
    super().__init__()

    self.hid_dim = hid_dim
    self.n_layers = n_layers

    self.embedding = nn.Embedding(input_dim, emb_dim)
    self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

    self.dropout = nn.Dropout(dropout)

  def forward(self, src):
    out = self.embedding(src)  # src = [src_len , batch_size]
    embedded = self.dropout(out) # embedded = [src_len, batch_size, emb_dim]
                                 # src를 dense 벡터에 매핑하는 임베딩 레이어의 출력
    out, (hidden, cell) = self.lstm(embedded)

    # out = [src_len, barch_size, hid_dim] / LSTM의 출력
    # hidden = [n_layer, batch_size, hid_dim] / LSTM의 hidden state
    # cell = [n_layer, batch_size, hid_dim] / LSTM의 cell state
    
    return hidden, cell 
    

---
### 2-2. Decoder
----

<img src='https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq3.png'>

- time step 당 단일 토큰을 출력 

- Decoder의 초기 hidden 및 cell state는 동일한 계층의 Encoder의 hidden 및 cell state인 context vector
1. 첫 번째 계층은 이전 time step의 hidden state를 받고

2. LSTM을 통해 현재 embedded 토큰으로 공급해 새로운 hidden 및 cell state를 생성

3. 다음 계층은 아래의 hidden state를 사용하며 이는 이전의 Layer의 hidden 및 cell state



In [15]:
class Decoder(nn.Module):
  def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
    super().__init__()

    self.output_dim = output_dim
    self.hid_dim = hid_dim 
    self.n_layers = n_layers

    self.embedding = nn.Embedding(output_dim, emb_dim)

    self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

    self.fc_layer = nn.Linear(hid_dim, output_dim)

    self.dropout = nn.Dropout(dropout)

  def forward(self, input, hidden, cell)  :
    
    # input = [batch_size]
    # hidden = [n_layer, batch_size, hid_dim]
    # cell = [n_layer, batch_size, hid_dim] 
    input = input.unsqueeze(0) # [1, batch_size]

    out = self.embedding(input) # embedded = [1, batch_size, emb_dim]
    embedded = self.dropout(out)

    output, (hidden, cell) = self.lstm(embedded, (hidden, cell))

    # Decoder의 Sequence_length는 각 time step에 대한 입력이 단일 word/torken만 처리하기 때문에 항상 값은 1이다.
    # output = [seq len, batch size, hid dim]   ->  [1, batch size, hid dim]
    # hidden = [n layers, batch size, hid dim]  ->  [n layers, batch size, hid dim]
    # cell = [n layers, batch size, hid dim]    ->  [n layers, batch size, hid dim]

    predict = self.fc_layer(output.squeeze(0)) # predict = [batch_size, output_dim]

    return predict, hidden, cell

---
### 2-3. Seq2Seq
---

<img src='https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq4.png'>

1. input 문장 받기

2. Encoder를 이용해 context 벡터 생성 

3. Decoder를 이용해 예측 output 문장 생성 

- assert 함수

  - 가정 설정문 

  - 디버깅 목적으로 사용, 일반적으로 입력 인수의 유효성을 검사하거나 프로그램의 내부 상태를 검증하는데 사용

  - 특정 조건이 충족되는지 확인하고, 조건이 True이면 아무것도 발생X, 미충족시 AssertionError가 발생

In [16]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()

    self.encoder = encoder
    self.decoder = decoder 
    self.device = device

    assert encoder.hid_dim == decoder.hid_dim, 'Hidden dimensions of encoder decoder must be equal'
    assert encoder.n_layers == decoder.n_layers, 'Encoder and decoder must have equal number of layers'


  def forward(self, src, trg, teacher_forcing_ratio = 0.5):
                  # src = [src_len, batch_size]
                  # trg = [trg_len, batch_size]

    batch_size = trg.shape[1]
    trg_len = trg.shape[0] # 타겟 토큰 길이 
    trg_vocab_size = self.decoder.output_dim # context vector의 차원
    
    # Decoder의 출력을 저장할 Tensor
    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)


    # Encoder의 Hidden state를 Decoder의 초기 hidden state로 사용
    hidden, cell = self.encoder(src)

    # Decoder의 첫 번째 input을 <token>으로 사용
    input = trg[0,:]


    '''한번에 batch_size만큼의 token들을 독립적으로 계산
    즉, 총 trg_len번의 for문이 돌아가며 이 for문이 다 돌아가야지만 하나의 문장이 decoding됨
    또한, 1번의 for문당 128개의 문장의 각 token들이 다같이 decoding되는 것'''
    
    for i in range(1, trg_len):
      
      output, hidden, cell = self.decoder(input, hidden, cell)

      # 예측값 저장 
      outputs[i] = output

      # teacher forcing 사용 여부
      teacher_force = random.random() < teacher_forcing_ratio

      # 가장 높은 확률 값 얻기
      top1 = output.argmax(1)

      # teacher_forcing의 경우 다음 LSTM에 target token 입력
      input = trg[i] if teacher_force else top1

    return outputs


---
# 3. Model setting
----

In [17]:
# 하이퍼 파라미터 지정
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)
enc_emb_dim = 256 # 임베딩 차원
dec_emb_dim = 256
hid_dim = 512 # hidden state 차원
n_layers = 4
enc_dropout = 0.5
dec_dropout = 0.5

In [28]:
# 모델 생성
enc = Encoder(input_dim, enc_emb_dim, hid_dim, n_layers, enc_dropout)
dec = Decoder(output_dim, dec_emb_dim, hid_dim, n_layers, dec_dropout)

model = Seq2Seq(enc, dec, device).to(device)

In [29]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (lstm): LSTM(256, 512, num_layers=4, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (lstm): LSTM(256, 512, num_layers=4, dropout=0.5)
    (fc_layer): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [30]:
# 가중치 초기화
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

# 모델의 학습가능한 파라미터 수 측정
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainableparameters')

The model has 22,303,493 trainableparameters


In [31]:
# optimizer
optimizer = optim.Adam(model.parameters())

# loss function
# pad에 해당하는 index는 무시합니다.
trg_pad_idx = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)

---
# 4. Train
---

In [32]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(tqdm(iterator)):
        src = batch.src.to(device)
        trg = batch.trg.to(device)
        optimizer.zero_grad()

        output = model(src,trg) # [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim) # loss 계산을 위해 1d로 변경
        trg = trg[1:].view(-1) # loss 계산을 위해 1d로 변경

        loss = criterion(output, trg)
        loss.backward()
        # 기울기 clip
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [33]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(tqdm(iterator)):
            src = batch.src.to(device)
            trg = batch.trg.to(device)

            # output: [trg len, batch size, output dim]
            output = model(src, trg, 0) # teacher forcing off
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim) # [(trg len -1) * batch size, output dim]
            trg = trg[1:].view(-1) # [(trg len -1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [34]:
num_epochs = 10
clip = 1

best_valid_loss = float('inf')

for epoch in range(num_epochs):
    
    train_loss = train(model, train_iterator, optimizer, criterion, clip)
    valid_loss = evaluate(model, valid_iterator, criterion)
      
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

  0%|          | 0/227 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

	Train Loss: 5.119 | Train PPL: 167.219
	 Val. Loss: 4.764 |  Val. PPL: 117.245


  0%|          | 0/227 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

	Train Loss: 3.589 | Train PPL:  36.192
	 Val. Loss: 4.183 |  Val. PPL:  65.581


  0%|          | 0/227 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

	Train Loss: 3.529 | Train PPL:  34.081
	 Val. Loss: 4.169 |  Val. PPL:  64.666


In [35]:
# best val loss일 때의 가중치를 불러옵니다.
model.load_state_dict(torch.load('tut1-model.pt'))

# test loss를 측정합니다.
test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

  0%|          | 0/8 [00:00<?, ?it/s]

| Test Loss: 4.182 | Test PPL:  65.473 |
